# Pro CS-GO Match Prediction

## TO-DO

- Figure out how to port data to a csv file. Need to figure out how to stack dictionaries on top of each other
- Parse data for each player individually


- Model will be trained on a bunch of team/player data at time of match

In [71]:
from bs4 import BeautifulSoup
import pandas as pd
import urllib3


class HTLV_Scrapper:
    
    def __init__(self):
        
        #self.soup = None
        self.domain = 'https://www.hltv.org'
#       self.ranking_url = 'https://www.hltv.org/ranking/teams/'
        self.ranking_url = self.domain + '/ranking/teams/'
        self.basic_parse = {}


    def getSoup(self, url):
        
        if 'hltv' not in url.split('.'):
            raise Exception('Given url, {}, is not in the HLTV domain'.format(url))
        
        try:
            http = urllib3.PoolManager()
            response = http.request('GET', url)
            return BeautifulSoup(response.data, 'html.parser')
            
        except Exception as e:
            print(str(e))
            raise e
        
        
    def get_basic_parse(self):
        
        soup = self.getSoup(self.ranking_url)
        for team in soup.findAll('div', class_='ranked-team standard-box'):
            temp_dict = {}
    
            # Find basic team info 
            name = (team.find('span', class_='name').text)
            rank = int(team.find('span', class_='position').text[1:])
            points = int(team.find('span', class_='points').text[1:-7])
            team_link = team.find('div', class_='more').find('a').get('href')
            temp_dict = {'name': name, 'rank': rank, 'points': points, 'team_link': team_link}
    
            # Find player links for each member of the team
            for i, player in enumerate(team.findAll('td', class_='player-holder')):
                temp_dict['player_link' + str(i)] = player.find('a').get('href')
        
            self.basic_parse[name] = temp_dict
        
        return self.basic_parse
    
    def get_match_history(self, team_name):
        
        if team_name in self.basic_parse:
            team_link = self.domain + self.basic_parse[team_name]['team_link']
            team_soup = self.getSoup(team_link)
            links = self.get_match_links(team_soup)
            
            for link in links:
                return self.get_match_history_helper(link)
                break #Only parse one link
        else:
            raise Exception('{} is not in top30 teams'.format(team_name))
    
    
    def get_match_history_helper(self, match):
        
        url = self.domain + match
        match_soup = self.getSoup(url)
        print(match_soup)
    
    
    def get_match_links(self, soup):
        
        table = soup.find('table',{'class':'match-table'})
        matches = table.find_all('td', {'class':'stats-button-cell'})
        match_links = [match.find('a').get('href') for match in matches]
        return match_links
        

    def parse_html_table(self, table):
            n_columns = 0
            n_rows=0
            column_names = []
    
            # Find number of rows and columns
            # we also find the column titles if we can
            for row in table.find_all('tr'):
                
                print(row)
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')
                if len(td_tags) > 0:
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())
    
            # Safeguard on Column Titles
            if len(column_names) > 0 and len(column_names) != n_columns:
                raise Exception("Column titles do not match the number of columns")
    
            columns = column_names if len(column_names) > 0 else range(0,n_columns)
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            row_marker = 0
            for row in table.find_all('tr'):
                column_marker = 0
                columns = row.find_all('td')
                for column in columns:
                    df.iat[row_marker,column_marker] = column.get_text()
                    column_marker += 1
                if len(columns) > 0:
                    row_marker += 1
                    
            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
            return df
        

        
#     def get_parse(self, ): 
#         return basic_parse_results

In [72]:
data_gen = HTLV_Scrapper()
temp = data_gen.get_basic_parse()

In [73]:
df = data_gen.get_match_history('Astralis')

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" id="metaViewport" name="viewport"/>
<meta content="1004164229" property="fb:admins"/>
<meta content="249997999009" property="fb:pages"/>
<meta content="1460388157605817" property="fb:app_id"/>
<meta content="DcypRFLQvgYQL5Acx7feoGWbblSsmKv6HpPI7mM_1uw" name="google-site-verification"/>
<link href="/img/static/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/img/static/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/img/static/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/img/static/favicon/manifest.json" rel="manifest"/>
<link color="#5bbad5" href="/img/static/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="#ffffff" name="theme-color"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,400i,700,700i|Oswald:700&amp;amp;subset=latin-e